In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# TODO
- See if ticket and cabin have any importance
- Try data imputation instead of fillna

# Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [ ]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

In [ ]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [ ]:
ds = torch.tensor(np.array(df))
df.shape, ds.shape

((891, 9), torch.Size([891, 9]))

# Model Creation

In [ ]:
num_epochs = 100
lr =.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, inp_size, h1, h2, h3, h4, h5, out_size):
        super(NeuralNet, self).__init__()
        self.inp_size = inp_size
        self.lay1 = nn.Linear(inp_size, h1)
        self.lay2 = nn.ReLU()
        self.lay3 = nn.Linear(h1, h2)
        self.lay4 = nn.ReLU()
        self.lay5 = nn.Linear(h2, h3)
        self.lay6 = nn.ReLU()
        self.lay7 = nn.Linear(h3, h4)
        self.lay8 = nn.ReLU()
        self.lay9 = nn.Linear(h4, h5)
        self.lay10 = nn.ReLU()
        self.lay11 = nn.Linear(h5, out_size)
        
    def forward(self,x):
        out = self.lay1(x)
        out = self.lay2(out)
        out = self.lay3(out)
        out = self.lay4(out)
        out = self.lay5(out)
        out = self.lay6(out)
        out = self.lay7(out)
        out = self.lay8(out)
        out = self.lay9(out)
        out = self.lay10(out)
        out = self.lay11(out)  # We don't apply sotmax the cross entropy loss will do that for us
        return out
    
model = NeuralNet(8,30,60,120,60,30,2).to(device)

In [ ]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr = lr)

# Training Model

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=True, batch_size=1) # Batch size of 1 sucks

In [ ]:
for epoch in range(num_epochs):
    avg_loss = 0
    for i,batch in enumerate(train_loader):

        y = batch[:,1].to(device).long()
        X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)

        # Forward pass
        output = model(X)
        
        # Backward pass
        loss = lossCategory(output, y)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()
        avg_loss += loss

        
    print(f"Epoch:{epoch+1}/{num_epochs} Loss:{avg_loss/len(train_loader)}")

Epoch:1/100 Loss:0.677807629108429
Epoch:2/100 Loss:0.6602651476860046
Epoch:3/100 Loss:0.649964451789856
Epoch:4/100 Loss:0.6288031339645386
Epoch:5/100 Loss:0.6191524267196655
Epoch:6/100 Loss:0.6152294874191284
Epoch:7/100 Loss:0.6134187579154968
Epoch:8/100 Loss:0.5974647998809814
Epoch:9/100 Loss:0.5876007676124573
Epoch:10/100 Loss:0.5874970555305481
Epoch:11/100 Loss:0.5874793529510498
Epoch:12/100 Loss:0.5934283137321472
Epoch:13/100 Loss:0.5644993782043457
Epoch:14/100 Loss:0.5709799528121948
Epoch:15/100 Loss:0.5554971694946289
Epoch:16/100 Loss:0.5404708981513977
Epoch:17/100 Loss:0.5318778157234192
Epoch:18/100 Loss:0.5285587906837463
Epoch:19/100 Loss:0.512500524520874
Epoch:20/100 Loss:0.5208219289779663
Epoch:21/100 Loss:0.5087610483169556
Epoch:22/100 Loss:0.5198904871940613
Epoch:23/100 Loss:0.49187222123146057
Epoch:24/100 Loss:0.495981901884079
Epoch:25/100 Loss:0.49757200479507446
Epoch:26/100 Loss:0.4945209324359894
Epoch:27/100 Loss:0.5080970525741577
Epoch:28/100

# Checking Accuracy

In [ ]:
correct = 0
for i,batch in enumerate(train_loader):
    y = batch[:,1].to(device).long().item()
    X =torch.cat((batch[:, :1], batch[:, 2:]), dim=1).to(device).to(torch.float32)
    output = model(X)
    softmax_output = F.softmax(output, dim=-1)
    y_pred = torch.argmax(softmax_output).item()
    if y_pred == y:
        correct += 1
print('Accuracy:', correct*100/i) 

Accuracy: 80.33707865168539


# Running Test Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))
test_loader = torch.utils.data.DataLoader(dataset=ds, shuffle=False, batch_size=1)

In [ ]:
ds.shape,df.shape

(torch.Size([418, 8]), (418, 8))

In [ ]:
l = []
id_ = []
for i,batch in enumerate(test_loader):
        X =batch.to(device).to(torch.float32)
        output = model(X)
        softmax_output = F.softmax(output, dim=-1)

        l.append(torch.argmax(softmax_output).item())
        id_.append(int(X[:,0].item()))

ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('/kaggle/working/submission.csv', index=False)
print("Saved new O/P")
ans

Saved new O/P


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
